# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Importing Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking current working directory
print(os.getcwd())

# Getting current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    
    file_path_list = glob.glob(os.path.join(root,'*'))

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
full_data_rows_list = [] 
    
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
# checking the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
from cassandra.cluster import Cluster
cluster = Cluster()

session = cluster.connect()

#### Create Keyspace

In [7]:
query = """
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
try:
    session.execute(query)
except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
try:
    session.set_keyspace("udacity")
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




# create table song, set key sessionId, itemInSession

In [9]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
# Let's start first by setting partition key - sessionId. In order for row to be unique let's set clustering column
# itemInSession
query = """create table if not exists song (artist text, song text, length float, sessionId int,
itemInSession int, primary key(sessionId, itemInSession))"""
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

# insert data into table

In [10]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into song (sessionId, itemInSession, artist, song, length) "
        query = query + "values(%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

# verifying data with select statement

In [11]:
try:
    rows = session.execute("select artist, song, length from song where sessionId = 338 and iteminsession = 4")
except Exception as e:
    print(e)
 
table_rows = []
for row in rows:
    table_row = (row.artist, row.song, row.length)
    table_rows.append(table_row)
df = pd.DataFrame(table_rows, columns=['artist', 'song', 'length'])
print(df.to_string(index=False))

artist                             song      length
Faithless  Music Matters (Mark Knight Dub)  495.307312


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

# creating table user_songs with userid, sessionId, itemInSession key

In [12]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
# Let's first start by partition key - userid, in order for row to be unique let's add clustering column
# sessionid and itemInSession in order for row to be unique and to sort by itemInSession
# if in one session there is only one user we can do query from first table writing only sessionid ommiting userid
query = """create table if not exists user_songs (userid int, artist text, song text, sessionId int,
itemInSession int, firstName text, lastName text, primary key(userid, sessionId, itemInSession))"""
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

# inserting data

In [13]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into user_songs (userid, sessionId, itemInSession, artist, song, firstName, lastName) "
        query = query + "values(%s, %s, %s, %s, %s, %s, %s) "
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

# verifying data with select statement

In [14]:
query = "select artist, song, firstName, lastName, itemInSession from user_songs where userid = 10 and sessionId = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
table_rows = []
for row in rows:
    table_row = (row.artist, row.song, row.firstname, row.lastname)
    table_rows.append(table_row)
df = pd.DataFrame(table_rows, columns=['artist', 'song', 'firstname', 'lastname'])
print(df.to_string(index=False))

artist                                               song firstname lastname
 Down To The Bone                                 Keep On Keepin' On    Sylvie     Cruz
     Three Drives                                        Greece 2000    Sylvie     Cruz
Sebastien Tellier                                          Kilometer    Sylvie     Cruz
    Lonnie Gordon  Catch You Baby (Steve Pitron & Max Sanna Radio...    Sylvie     Cruz


# creating table user with song, userid key

In [15]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
# unique value has to be by song and users, partition has to be song because in where statements there will be par
# ticular song
query = """create table if not exists user (userid int, firstName text, lastName text, song text, 
primary key(song, userid))"""
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

# inserting data

In [16]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into user (song, userid, firstName, lastName) "
        query = query + "values(%s, %s, %s, %s) "
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

# verifying data with select statement

In [17]:
query = "select userid, firstName, lastName from user where song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
table_rows = []
for row in rows:
    table_row = (row.firstname, row.lastname)
    table_rows.append(table_row)
df = pd.DataFrame(table_rows, columns = ['firstname', 'lastname'])
print(df.to_string(index=False))

firstname lastname
Jacqueline    Lynch
     Tegan   Levine
      Sara  Johnson


### Drop the tables before closing out the sessions

In [18]:
try:
    session.execute("drop table user")
except Exception as e:
    print(e)

try:
    session.execute("drop table user_songs")
except Exception as e:
    print(e)
    
try:
    session.execute("drop table song")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()